# Import the necessary modules

pip install git+https://github.com/AmeriFinn/Python-Projects/blob/Equity-Portfolio-Analysis/EquityAnalysis.py

In [1]:
## Standard Python modules
import math
from numpy import random
from datetime import *
import pandas as pd          

from EquityAnalysis import * ## Module written by me for collecting price data
import plotly.express as px  ## Alternate plotting module than matplotlib

## Modules needed for widgets in jupyter notebooks
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import *

## Expand the cell length to leave less empty space in the margins
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Hello World


In [ ]:
import datapane as dp
from datetime import date

dp.login(token="f29b5ea0db584bee2a4a0d161868caf16c351e1c")

# Start analysis of my simple trading strategy using `Daily Binary Options` on the underlying contract for the `S&P 500 Front Month Futures Contract`

## Strategy involves:
## &emsp;(1) At 9am every day, buy the `call` with strike closest to spot. 
### &emsp; This will be the contract with the price closest to 50, implying a 50% chance of finishing ITM.
## &emsp;(2) Regardless of what happens that day, hold the contract until expiration at 2:15pm.
## &emsp;(3-a) If the futures price expires `above` the strike price, I am `paid 100` for a `profit of ~50` per contract.
## &emsp;(3-b) If the futures price expires `below` the strike price, I am `paid 0` for a `loss of ~50` that I paid for the contract
## &emsp;(4) Repeat this process every day with an increasing number of contracts purchased depending on the available cash balance of the portfolio.

In [2]:
## Collect price data - include YM=F ticker for unresolved bug in EA.Data
portfolio = Data(tickers  = ["ES=F", "YM=F"], 
                 shares   = [1],
                 period   = "max",
                 interval = "1d")
portfolio.Collect("prices")
prices = portfolio.PriceData

## Parse out price data for all of 2020
prices = prices[(prices.index.year == 2020)] # & \
#                (prices.index.month > 5) & \
#                (prices.index.month < 13)]

## Seperate out price data needed for S&P 500 Futures Front Month contract
binaries = prices.loc[:,idx[['Open', 'Close'], 'ES=F']].copy()
binaries.columns = ['Open', 'Close']

[*********************100%***********************]  2 of 2 completed


In [ ]:
prices.index.year.unique()

## Define UDF's needed to analyze strategy results

In [ ]:
def WinOrLose(Open, Close, AvgS2S=0):
    if Close > (Open+AvgS2S):
        return 'Win'
    else:
        return 'Lose'
    

In [ ]:
def GainOrLoss(Open, Close, AvgCost=50, AvgS2S=0):
    ## Determine if the close price was above or below
    ## the open price + the basis between the strike price and open price.
    ## A positive AvgS2S implies a contract is purchased on average with a 
    ## strike price greater than the opening spot price. A negative AvgS2S
    ## implies a contract is purchased on average with a strike price less
    ## than the opening spot price.
    if Close > (Open+AvgS2S):
        # Payoff of $100, less avg. cost of contract  
        # & $2 of NADEX fees
        return 100 - AvgCost - 2 
    else:
        # Payoff of $0, less avg. cost of contract
        # & $1 of NADEX fees
        return -AvgCost - 1 

In [82]:
def MyContracts(CashBal, AvgCost=50, InitDep=250):
    if CashBal < AvgCost:
        # If there isn't enough cash to buy even 1 contract, 
        # the strategy has lost all of the initial $250 posted
        return 0
    
    elif CashBal < 300:
        # If current balance is below the minimum initial deposit
        # on NADEX, only purchase 1 contract
        return 1

    elif CashBal < 1250:
        # There is surplus cash and more than 1 contract can be purchased.
        # The number of extra contracts purchased will be calculated as
        # the surplus cash divided by the average cost of a contract.
        # The max contracts that will be purchased when CashBal < $1,250 is 10
        return 1 + max(min(math.floor((CashBal - 500) / 50),9),1)
        #return 1 + min()    
    
    elif CashBal < 5000:
        # There is surplus cash and more than 1 contract can be purchased
        # The max contracts that will be purchased when CashBal < $5,000 is 15
        return 10 + max(min(math.floor(5 * (CashBal - 1500) / 50), 5), 1)

    elif CashBal < 10000:
        # There is surplus cash and more than 1 contract can be purchased
        # When CashBal < $10,000, exactly 30 contracts will be purchased.
        #return 15 + math.floor(((CashBal  - 5000) / 5000)*15)
        return 30
    
#    elif CashBal < 25000:
#        return 45
    
    elif CashBal < 50000:
        # There is surplus cash and more than 1 contract can be purchased
        # When CashBal < $50,000, exactly 60 contracts will be purchased.
        #return 30 + math.floor((30 * (CashBal  - 10000) / 40000))
        return 60
    
    elif CashBal < 100000:
        # There is surplus cash and more than 1 contract can be purchased
        # When CashBal < $100,000, exactly 85 contracts will be purchased.
        return 85
    
    else:
        # There is surplus cash and more than 1 contract can be purchased
        # When CashBal > $100,000, exactly 100 (the max) contracts will be purchased.
        return 100

In [91]:
def MyPnL(df, Withdrawals=False, CashLimit=75000, CashReset = 25000, AvgCost=50, AvgS2S=0, InitDep=250):
    
    # Apply lambda function to determine if a $50 call would've won or lost on each day
    df.loc[:,'WinOrLose'] = df.apply(lambda x: WinOrLose(x.Open,x.Close), axis=1)
    df.loc[:,'GainOrLoss'] = df.apply(lambda x: GainOrLoss(x.Open,x.Close,AvgCost,AvgS2S), axis=1)
    
    # Add starting balances and contract amounts
    df.loc[df.index[0],'Contracts']   = 1
    df.loc[df.index[0],'OneDayPnL']   = 0
    df.loc[df.index[0],'CashBal']     = InitDep
    df.loc[df.index[0],'Deposits']    = InitDep
    df.loc[df.index[0],'Withdrawals'] = 0
    
    # Calculate PnL for day 1
    df.loc[df.index[0],'OneDayPnL'] = df.loc[df.index[0],'GainOrLoss']
    df.loc[df.index[0],'CashBal']  += df.loc[df.index[0],'OneDayPnL']
    
    # Initiate loop through the remaining dates to calculate
    # the number of contracts that could be purchased and the
    # resulting PnL and cash balance
    for i in range(1,df.shape[0]):
        
        # Calculate the number of Contracts that can be afforded
        df.loc[df.index[i],'Contracts'] = MyContracts(df.loc[df.index[i-1],'CashBal'])
        # Calculate resulting PnL
        df.loc[df.index[i],'OneDayPnL'] = df.loc[df.index[i],'Contracts'] * \
                                          df.loc[df.index[i],'GainOrLoss']
        # Calculate the new Cash Balance
        df.loc[df.index[i],'CashBal']   = df.loc[df.index[i-1],'CashBal'] + \
                                          df.loc[df.index[i],'OneDayPnL']
        
        # Determine if a deposit is needed
        if df.loc[df.index[i],'CashBal'] <= AvgCost:
            # If the cash balance is not enough to buy 1 contract,
            # re-contribute the selected initial investment of cash to the portfolio
            
            # Record the contribution
            df.loc[df.index[i],'Deposits'] = df.loc[df.index[i-1],'Deposits'] + InitDep
            # Increase the Cash Balance
            df.loc[df.index[i],'CashBal']  = df.loc[df.index[i-1],'CashBal'] + InitDep
            
        else:
            
            # Otherwise, the amount deposited to the portfolio has not changed
            df.loc[df.index[i],'Deposits'] = df.loc[df.index[i-1],'Deposits']
        
        # Determine if a withdrawal should be made
        if (df.loc[df.index[i],'CashBal'] > CashLimit) & (Withdrawals == True) & (df.loc[df.index[i],'WinOrLose'] == 'Win'):
            # If the cash balance exceeds `CashLimit`, I want to make a withdrawal and "reset"
            
            ## Alternate method to below section that is commented out
            # Only withdraw $1k if the CashBal > CashLimit. Theory is
            # this could generate $1k of income per day once the CashBal
            # reaches a certain limit
            df.loc[df.index[i], 'Withdrawals'] = df.loc[df.index[i-1],'Withdrawals'] + CashReset
            df.loc[df.index[i],'CashBal']     -= CashReset
            
            #df.loc[df.index[i],'Withdrawals'] = df.loc[df.index[i-1],'Withdrawals'] + \
            #                                    df.loc[df.index[i],'CashBal'] - \
            #                                    CashReset
            #df.loc[df.index[i],'CashBal']     = CashReset
        else:
            # Otherwise, the amount earned is not large enough to warrant a withdrawal
            df.loc[df.index[i],'Withdrawals'] = df.loc[df.index[i-1],'Withdrawals']
            
    
    # Calculate the Net Postion over time
    df.loc[:,'Net PnL']       = df['CashBal'] - df['Deposits'] + df['Withdrawals']
    df.loc[:,'Profit Margin'] = round(100 * df['Net PnL'] / df['Deposits'], 2)
    
    return df

## Plot out the relation between my cash balance and the number of shares I will purchase

def MyContracts(CashBal, AvgCost=50, InitDep=250):
    return 1

In [75]:
def MyContracts(CashBal, AvgCost=50, InitDep=250):
    if CashBal < AvgCost:
        return 0
    elif CashBal < 300:
        return 1
    
    elif CashBal < 1500:
        return max(min(math.floor(CashBal * .20 / AvgCost), 100), 1)

    elif CashBal < 5000:
        return max(min(math.floor(CashBal * .30 / AvgCost), 100), 1)
        
    elif CashBal < 15000:
        return max(min(math.floor(CashBal * .35 / AvgCost), 100), 1)

    elif CashBal < 25000:
        return max(min(math.floor(CashBal * .40 / AvgCost), 100), 1)
    
    elif CashBal < 50000:
        return max(min(math.floor(CashBal * .40 / AvgCost), 100), 1)
    elif CashBal < 100000:
        return max(min(math.floor(CashBal * .50 / AvgCost), 100), 1)
    
    else:
        return max(min(math.floor(CashBal * .50 / AvgCost), 100), 1)
        
        #return max(min((CashBal - 300) / (AvgCost * 4),100),1)
#    elif CashBal < 5000:
#        return (CashBal - )


In [83]:
CashDict = {}
i = 0

for n in range(50,125000,50):
    CashBal = n
    Contrac = MyContracts(CashBal)
    PctPrtf = (Contrac * 50) / CashBal
    
    CashDict[i] = [CashBal, Contrac, round(PctPrtf*100,2)]
    i += 1
    
dfCon = pd.DataFrame.from_dict(CashDict, 
                               orient='index', 
                               columns=['Cash Balance', 'Contracts', '% of Portfolio'])
dfCon.head()

fig = px.line(data_frame=dfCon,
              x = 'Cash Balance',
              y = ['Contracts', '% of Portfolio'],
              title = 'Binary Option Contracts Purchased Dependent on Cash Balance',
              labels=dict(value=""))

fig.update_layout(hovermode="x unified")
fig.show()

## Use the UDF's to analyze this strategy over the course of 2020

In [94]:
## Use `MyPnL` to calculate the results of my trading strategy in 2020
binaries = MyPnL(binaries, Withdrawals=True, CashLimit=20000, CashReset = 1000, InitDep=500)

## Display the top and bottom 5 rows of the results table
display(binaries.head(5).append(binaries.tail(5)))
print(binaries.shape)

## Calculate and print the Wins, Losses, and Win/Loss ratio
Wins = binaries[binaries.WinOrLose == 'Win'].shape[0]
Loss = binaries.WinOrLose.shape[0] - Wins 

print(f"\nWins: {Wins}")
print(f"Losses: {Loss}")
print(f"Win-Lose Ratio: {round(100*Wins/Loss,2)}%")

,Open,Close,WinOrLose,GainOrLoss,Contracts,OneDayPnL,CashBal,Deposits,Withdrawals,Net PnL,Profit Margin
Date,,,,,,,,,,,
2020-01-02,3237.00,3259.00,Win,48,1.0,48.0,548.0,500.0,0.0,48.0,9.6
2020-01-03,3261.00,3235.50,Lose,-51,2.0,-102.0,446.0,500.0,0.0,-54.0,-10.8
2020-01-06,3220.25,3243.50,Win,48,2.0,96.0,542.0,500.0,0.0,42.0,8.4
2020-01-07,3243.50,3235.25,Lose,-51,2.0,-102.0,440.0,500.0,0.0,-60.0,-12.0
2020-01-08,3231.75,3260.25,Win,48,2.0,96.0,536.0,500.0,0.0,36.0,7.2
2020-12-23,3674.25,3681.50,Win,48,15.0,720.0,2847.0,1000.0,13000.0,14847.0,1484.7
2020-12-28,3682.00,3727.50,Win,48,15.0,720.0,3567.0,1000.0,13000.0,15567.0,1556.7
2020-12-29,3731.00,3720.00,Lose,-51,15.0,-765.0,2802.0,1000.0,13000.0,14802.0,1480.2
2020-12-30,3724.50,3724.25,Lose,-51,15.0,-765.0,2037.0,1000.0,13000.0,14037.0,1403.7


(251, 11)

Wins: 140
Losses: 111
Win-Lose Ratio: 126.13%


## Plot the results of the trading strategy for `2020`

In [95]:
fig = px.line(data_frame = binaries,
              y = ['CashBal', 'Contracts', 'Deposits', 'Withdrawals', 'Net PnL', 'Profit Margin'],
              log_y=True)
             #width = 1000,
             #height = 700)
fig.update_layout(hovermode="x unified")
fig.show()

## Set up widgets to quickly create visuals for given time frames

In [19]:
style = {'description_width': 'max-content'}

Year1Drop = widgets.Dropdown(options = range(2021,2000,-1),
                            value = 2020,
                            disabled=False,
                            description='Start Year:',
                            style = style)

Year2Drop = widgets.Dropdown(options = range(2021,2000,-1),
                            value = 2020,
                            disabled=False,
                            description='End Year:',
                            style = style)

WdrwCheck = widgets.Checkbox(Value=False,
                            description="Make Withdrawals?",
                            disabled=False,
                            indent=False,
                            style=style)

LimitInt = widgets.IntSlider(value=75000,
                                  min=5000,
                                  max=150000,
                                  step=5000,
                                  description='Cash Balance Limit:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

ResetInt = widgets.IntSlider(value=7500,
                                  min=1000,
                                  max=110000,
                                  step=1000,
                                  description='Reset Cash Amount:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

AvCstInt = widgets.IntSlider(value=50,
                                  min=35,
                                  max=65,
                                  step=1,
                                  description='Average Contract Cost:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

AvBasInt = widgets.IntSlider(value= 0,
                             min  = -10,
                             max  = 10,
                             step = 1,
                             description = 'Average (Strike - Spot):',
                             continuous_update=False,
                             orientation='horizontal',
                             readout=True,
                             readout_format='d',
                             style=style
                             )

IniDepIt = widgets.IntSlider(value=250,
                                  min=250,
                                  max=5000,
                                  step=50,
                                  description='Initial Deposit:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

PbshCheck = widgets.Checkbox(Value=False,
                            description="Publish Report?",
                            disabled=False,
                            indent=False,
                            style=style)

GoButton = widgets.Button(value = False,
                         description='Plot Timeframe',
                         disabled = False,
                         button_style='success',
                         tooltip='Description',
                         icon='check',
                         style=style)

def click_button(b):
    
    ## Collect input values
    StartYear = Year1Drop.value
    EndYear   = Year2Drop.value
    Checkbox  = WdrwCheck.value
    CashLimit = LimitInt.value
    ResetAmnt = ResetInt.value
    AvgCost   = AvCstInt.value
    AvgS2S    = AvBasInt.value
    InitDepos = IniDepIt.value
    Publish   = PbshCheck.value
    
    ## Clear previous output
    clear_output(wait=False)
    
    ## Parse out price data for all of 2020
    df = prices[(prices.index.year >= StartYear) & \
                (prices.index.year <= EndYear)]

    ## Seperate out price data needed for S&P 500 Futures Front Month contract
    df = df.loc[:,idx[['Open', 'Close'], 'ES=F']]
    df.columns = ['Open', 'Close']
    
    ## Calculate the PnL over the selected time frame
    df = MyPnL(df, 
               Withdrawals = Checkbox, 
               CashLimit   = CashLimit, 
               CashReset   = ResetAmnt, 
               AvgCost     = AvgCost, 
               AvgS2S      = AvgS2S, 
               InitDep     = InitDepos)
    
    ## Display the input widgets
    display(Year1Drop, Year2Drop, IniDepIt, AvCstInt, AvBasInt, WdrwCheck, LimitInt, ResetInt, PbshCheck, GoButton)
    
    ## Reset values to the user provided inputs
    Year1Drop.value = StartYear
    Year2Drop.value = EndYear
    WdrwCheck.value = Checkbox
    LimitInt.value  = CashLimit
    ResetInt.value  = ResetAmnt
    AvCstInt.value  = AvgCost
    AvBasInt.value  = AvgS2S
    IniDepIt.value  = InitDepos
    
    ## Create and show the line plot
    fig = px.line(data_frame = df,
                  y = ['CashBal', 'Contracts', 'Deposits', 'Withdrawals', 'Net PnL', 'Profit Margin'],
                  log_y = True,
                  title = "Results of Gral's Binary Options Discrete Piece-Wise Trading Strategy | " + \
                          f"{StartYear}-{EndYear}",
                  width = 1000,
                  height = 700)
    fig.update_layout(hovermode="x unified")
    
    descDict = {}
    descDict['Avg. Contract Cost'] = f"${AvgCost}"
    descDict['Initial Deposit']    = f"${InitDepos}"
    descDict['Withdrawals?']       = Checkbox
    if Checkbox == True:
        descDict['Max Balance']    = f"${CashLimit}"
        descDict['Cash Reset Amt'] = f"${ResetAmnt}"
    
    display(descDict)
    fig.show()
    
    if Publish == True:
        # Create report
        r = dp.Report(
            f"Results of Gral's Binary Option Strategy | {StartYear}-{EndYear} \n{descDict}",
            f'_Built using data from [Yahoo Finance](https://finance.yahoo.com/) on {date.today()}_',
            dp.Plot(fig),
            dp.DataTable(df),
        )

        # Publish
        r.publish(name=f"Results of Gral's Binary Option Strategy | {StartYear}-{EndYear}", 
                  open=True, 
                  description=f'{descDict}')

    ## Display the top and bottom 3 rows of the results data set
    display(df.head(3).append(df.tail(3)))
    print(df.shape)

GoButton.on_click(click_button)

In [21]:
## Collect price data - include AAPL ticker for unresolved bug in EA.Data
portfolio = Data(tickers  = ["ES=F", "AAPL"], 
                 shares   = [1],
                 period   = "max",
                 interval = "1d")
portfolio.Collect("prices")
prices = portfolio.PriceData

[*********************100%***********************]  2 of 2 completed


## Create the interactive visuals

In [96]:
fig = VBox([Year1Drop, Year2Drop, IniDepIt, AvCstInt, AvBasInt, WdrwCheck, LimitInt, ResetInt, PbshCheck, GoButton])
display(fig)

Dropdown(description='Start Year:', index=2, options=(2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 20…

Dropdown(description='End Year:', index=1, options=(2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012…

IntSlider(value=500, continuous_update=False, description='Initial Deposit:', max=5000, min=250, step=50, styl…

IntSlider(value=50, continuous_update=False, description='Average Contract Cost:', max=65, min=35, style=Slide…

IntSlider(value=1, continuous_update=False, description='Average (Strike - Spot):', max=10, min=-10, style=Sli…

...

Checkbox(value=False, description='Publish Report?', indent=False, style=DescriptionStyle(description_width='m…

Button(button_style='success', description='Plot Timeframe', icon='check', style=ButtonStyle(), tooltip='Descr…

{'Avg. Contract Cost': '$50',
 'Initial Deposit': '$500',
 'Withdrawals?': True,
 'Max Balance': '$30000',
 'Cash Reset Amt': '$1000'}

,Open,Close,WinOrLose,GainOrLoss,Contracts,OneDayPnL,CashBal,Deposits,Withdrawals,Net PnL,Profit Margin
Date,,,,,,,,,,,
2019-01-02,2508.00,2511.00,Win,48,1.0,48.0,548.0,500.0,0.0,48.0,9.60
2019-01-03,2482.25,2447.75,Lose,-51,2.0,-102.0,446.0,500.0,0.0,-54.0,-10.80
2019-01-04,2451.75,2531.25,Win,48,2.0,96.0,542.0,500.0,0.0,42.0,8.40
2020-12-29,3731.00,3720.00,Lose,-51,1.0,-51.0,155.0,2000.0,0.0,-1845.0,-92.25
2020-12-30,3724.50,3724.25,Lose,-51,1.0,-51.0,104.0,2000.0,0.0,-1896.0,-94.80
2020-12-31,3725.00,3748.75,Win,48,1.0,48.0,152.0,2000.0,0.0,-1848.0,-92.40


(505, 11)


## Plot S&P 500 E-Mini Daily Close

In [ ]:
ClosePrices = prices.loc[:,idx[['Open','Close'], 'ES=F']]
ClosePrices = ClosePrices[ClosePrices.index.year > 2017]
ClosePrices.columns = ['SP500 E-Mini Open', 'SP500 E-Mini Close']
#temp.head()

In [ ]:
fig = px.line(data_frame = ClosePrices,
              y = 'SP500 E-Mini Close',
              title = 'S&P500 E-Mini Front Month Daily Close - 2020')
fig.update_traces(line_color = '#147852')
fig.show()

In [ ]:
ClosePrices.loc[:,'SP500 E-Mini Close'] = \
ClosePrices['SP500 E-Mini Close'] - ClosePrices['SP500 E-Mini Open']

In [ ]:
def GoLColor(Close):
    if Close >= 0:
        return '+'
    else:
        return '-'

In [ ]:
ClosePrices.loc[:,'Colors'] = ClosePrices.apply(lambda x: GoLColor(x['SP500 E-Mini Close']), axis=1)

In [ ]:
ClosePrices.head()

In [ ]:
ClosePrices.loc[:,'PctChange'] = ClosePrices['SP500 E-Mini Close'] / ClosePrices['SP500 E-Mini Open']

In [ ]:
bar = px.bar(data_frame = ClosePrices,
             y = 'SP500 E-Mini Close', 
             base='SP500 E-Mini Open',color='Colors')
bar.show()

## Plot Annual Win/Loss Stats

In [ ]:
WinLoss = prices.loc[:,idx[:, 'ES=F']][['Open','Close']]
WinLoss.columns = ['Open', 'Close']
WinLoss.loc[:,'WinOrLose'] = WinLoss.apply(lambda x: WinOrLose(x.Open,x.Close), axis=1)
WinLoss.loc[:,'Year'] = WinLoss.index.year
WinLoss = WinLoss[(WinLoss.Year > 2000) & (WinLoss.Year < 2021)]

WinLoss.head(3).append(WinLoss.tail(3))

In [ ]:
yearDict = {}
for year in WinLoss.Year.unique():
    dft = WinLoss[WinLoss.Year == year]
    
    Wins = dft[dft.WinOrLose == 'Win'].shape[0] / dft.shape[0]
    Loss = (dft.WinOrLose.shape[0] - dft[dft.WinOrLose == 'Win'].shape[0]) / dft.shape[0]

    yearDict[year] = [round(100*Wins,2), round(100*Loss,2), round(100*Wins/Loss,2)]

WinLoss.head(3).append(WinLoss.tail(3))

In [ ]:
WinLoss1 = pd.DataFrame.from_dict(yearDict, orient='index', columns=['Gains','Losses','Win Ratio'])
WinLoss1.head(3).append(WinLoss1.tail(3))

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = px.bar(data_frame = WinLoss1,
             y = ['Gains', 'Losses'],
             title = 'S&P 500 E-Mini Annual 1-Day Gains Or Losses',
             labels=dict(index="Year",value="Daily Gains Or Losses"))

fig.add_shape(type='line',
             x0=2000, x1=2022, y0=50,y1=50)

fig.show()

## Set up for Monte Carlo Simulations
    - Establish # of iterations and assumption parameters
    - Define functions to create random samples and collect summary stats
    - 
    

In [ ]:
iterations = 10000

#mu = ClosePrices['SP500 E-Mini Close'].mean()
#sd = ClosePrices['SP500 E-Mini Close'].std()

In [ ]:
mu = ClosePrices['PctChange'].mean() * 100
sd = ClosePrices['PctChange'].std()  * 100

In [ ]:
mu, sd

### Define functions

In [ ]:
def RandPrices(mu, sd, Len=252, Base=3500):
    Open = {}
    Clos = {}
    mkts = random.normal(loc=mu, scale= sd, size=Len)/100
    afmk = random.normal(loc=mu/3, scale= sd/3, size=Len)/100
    
    Open[0] = float(Base * (1 + afmk[0]))
    Clos[0] = float(Open[0] * (1 + mkts[0]))
    
    for i in range(1, Len):
        Open[i] = float(Clos[i-1] * (1 + afmk[i]))
        Clos[i] = float(Open[i] * (1 + mkts[i]))
    
    Open, Clos = pd.Series(Open, name='Open'), \
                 pd.Series(Clos, name='Close')
    
    df = pd.concat([Open, Clos], axis=1)
    df.loc[:,'Height'] = df.Close - df.Open
    df.loc[:,'Colors'] = df.apply(lambda x: GoLColor(x['Height']), axis=1)
    
    df.index = [datetime.today() + timedelta(days=n) for n in df.index]
    
    return df

In [ ]:
px.bar(data_frame=RandPrices(0.1,1.75), y='Height', base='Open', color='Colors')

In [ ]:
def MonteCarlo(mu, sd, Samples=1000, Len=252, Base=3500, 
               Withdraw=False, CashLimit=75000, CashReset = 25000, AvgCost=50, AvgS2S=0, InitDep=250):
    
    ## Create storage lists
    Storage = []
    
    for i in range(Samples):
        clear_output(wait=True)
        print(f"Simulation - {i+1}/{Samples}")
        print(f"Withdrawals?: {Withdraw}, Cash Limit: {CashLimit}, Cash Reset: {CashReset},Strike - Spot: {AvgS2S}, Avg Cost: {AvgCost}, Init Deposit: {InitDep}")
        
        if i >= 1:
            if df.shape[0] <= 5:
                display(df)
            else:
                display(df.describe())
        
        # Create the random sample
        prices = RandPrices(mu=mu, 
                            sd=sd,
                            Len=Len,
                            Base=Base)
        temp = MyPnL(prices, 
                     Withdrawals = Withdraw, 
                     CashLimit   = CashLimit, 
                     CashReset   = CashReset, 
                     AvgCost     = AvgCost, 
                     AvgS2S      = AvgS2S, 
                     InitDep     = InitDep)
        
        # Collect the final results of employing 
        # this strategy over the desired timespan
        results = temp.iloc[-1]
        
        # Append results to storage lists
        Storage.append([
                        results.CashBal, 
                        results.Deposits, 
                        results.Withdrawals, 
                        results['Net PnL'],
                        results['Profit Margin']
                       ])
        
        df = pd.DataFrame(Storage,
                          columns=['Cash Balance', 'Deposits', 'Withdrawals', ' Net PnL', 'Profit Margin'])
        
    
    clear_output(wait=True)
    display(df.describe())
    return df

In [ ]:
mu, sd

In [ ]:
df = MonteCarlo(mu=mu,sd=sd, Samples=25,Withdraw=True, CashLimit=25000, CashReset = 25000, AvgCost=54, AvgS2S=-2, InitDep=2500)

In [ ]:
hist = px.histogram(data_frame=df, marginal='rug', nbins=10)
hist.show()


### Create widgets to visualize Monte Carlo outcomes

In [ ]:
style = {'description_width': 'max-content'}

ItersIntMC = widgets.IntSlider(value=250,
                                  min=10,
                                  max=5000,
                                  step=10,
                                  description='Samples:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

WdrwCheckMC = widgets.Checkbox(Value=False,
                            description="Make Withdrawals?",
                            disabled=False,
                            indent=False,
                            style=style)

LimitIntMC = widgets.IntSlider(value=75000,
                                  min=10000,
                                  max=150000,
                                  step=5000,
                                  description='Cash Balance Limit:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

ResetIntMC = widgets.IntSlider(value=7500,
                                  min=1000,
                                  max=110000,
                                  step=1000,
                                  description='Reset Cash Amount:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

AvCstIntMC = widgets.IntSlider(value=50,
                                  min=35,
                                  max=65,
                                  step=1,
                                  description='Average Contract Cost:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

AvBasIntMC = widgets.IntSlider(value= 0,
                             min  = -10,
                             max  = 10,
                             step = 1,
                             description = 'Average (Strike - Spot):',
                             continuous_update=False,
                             orientation='horizontal',
                             readout=True,
                             readout_format='d',
                             style=style
                             )

IniDepItMC = widgets.IntSlider(value=250,
                                  min=250,
                                  max=5000,
                                  step=50,
                                  description='Initial Deposit:',
                                  continuous_update=False,
                                  orientation='horizontal',
                                  readout=True,
                                  readout_format='d',
                                  style=style
                            )

PbshCheckMC = widgets.Checkbox(Value=False,
                            description="Publish Report?",
                            disabled=False,
                            indent=False,
                            style=style)

GoButtonMC = widgets.Button(#value = False,
                         description='Plot Timeframe',
                         disabled = False,
                         button_style='success',
                         tooltip='Description',
                         icon='check',
                         style=style)

In [ ]:
def click_button(b):
    
    ## Collect input values
    Iterats     = ItersIntMC.value
    CheckboxMC  = WdrwCheckMC.value
    CashLimitMC = LimitIntMC.value
    ResetAmntMC = ResetIntMC.value
    AvgCostMC   = AvCstIntMC.value
    AvgS2SMC    = AvBasIntMC.value
    InitDeposMC = IniDepItMC.value
    PublishMC   = PbshCheckMC.value
    
    ## Clear previous output
    clear_output(wait=False)
    
    ## Generate random samples
    df = MonteCarlo(mu          = mu,
                    sd          = sd, 
                    Samples     = Iterats,
                    Withdraw    = CheckboxMC, 
                    CashLimit   = CashLimitMC, 
                    CashReset   = ResetAmntMC, 
                    AvgCost     = AvgCostMC, 
                    AvgS2S      = AvgS2SMC, 
                    InitDep     = InitDeposMC)

    ## Reset values to the user provided inputs
    ItersIntMC.value  = Iterats
    WdrwCheckMC.value = CheckboxMC
    LimitIntMC.value  = CashLimitMC
    ResetIntMC.value  = ResetAmntMC
    AvCstIntMC.value  = AvgCostMC
    AvBasIntMC.value  = AvgS2SMC
    IniDepItMC.value  = InitDeposMC
    
    ## Create and show the line plot
    hist = px.histogram(data_frame=df, marginal='rug', nbins=2000)
    hist.show()
    
    #fig = px.line(data_frame = df,
    #              y = ['CashBal', 'Contracts', 'Deposits', 'Withdrawals', 'Net PnL', 'Profit Margin'],
    #              log_y = True,
    #              title = "SIMULATION Results of Gral's Binary Options Discrete Piece-Wise Trading Strategy",
    #              width = 1000,
    #              height = 700)
    #fig.update_layout(hovermode="x unified")
    
    descDict = {}
    descDict['Avg. Contract Cost'] = f"${AvgCostMC}"
    descDict['Initial Deposit']    = f"${InitDeposMC}"
    descDict['Withdrawals?']       = CheckboxMC
    if Checkbox == True:
        descDict['Max Balance']    = f"${CashLimitMC}"
        descDict['Cash Reset Amt'] = f"${ResetAmntMC}"
    
    display(descDict)
    print(df.shape)
    #fig.show()
    
    #bar = px.bar(data_frame = df,
    #         y = 'Height', 
    #         base='Open',color='Colors')
    #bar.show()
    
    if PublishMC == True:
        # Create report
        r = dp.Report(
            f"Results of Gral's Binary Option Strategy \n{descDict}",
            f'_Built using data from [Yahoo Finance](https://finance.yahoo.com/) on {date.today()}_',
            dp.Plot(hist),
            dp.DataTable(df),
        )

        # Publish
        r.publish(name=f"Results of Gral's Binary Option Strategy", 
                  open=True, 
                  description=f'{descDict}')
    
    ## Display the input widgets
    display(ItersIntMC, IniDepItMC, AvCstIntMC, 
            AvBasIntMC, WdrwCheckMC, LimitIntMC, 
            ResetIntMC, PbshCheckMC, GoButtonMC)
    
    
GoButtonMC.on_click(click_button)

In [ ]:
fig = VBox([ItersIntMC, IniDepItMC, AvCstIntMC, 
            AvBasIntMC, WdrwCheckMC, LimitIntMC,
            ResetIntMC, PbshCheckMC, GoButtonMC])
display(fig)

In [ ]:
print(f"mu, sd = {mu, sd}")

In [ ]:
test = RandPrices(mu=mu, sd=sd)
test = MyPnL(test)

## Create and show the line plot
fig = px.line(data_frame = test,
              y = ['CashBal', 'Contracts', 'Deposits', 'Withdrawals', 'Net PnL', 'Profit Margin'],
              log_y = True,
              title = "SIMULATION Results of Gral's Binary Options Discrete Piece-Wise Trading Strategy",
              width = 1000,
              height = 700)
fig.update_layout(hovermode="x unified")
fig.show()

bar = px.bar(data_frame = test,
             y = 'Height', 
             base='Open',color='Colors')
bar.show()

display(test.head(5).append(test.tail(5)))

In [ ]:
Samples    = 1000
Len        = 252
Base       = 3600
CashLimit  = [c for c in range(10000,20000,1000)] + [c for c in range(20000,50001,2500)]
CashReset  = CashLimit
AvgCost    = [c for c in range(45,56,1)]
Strik2Spot = [c for c in range(-5,6,1)]
InitDepos  = [d for d in range(250,2500,250)] + [d for d in range(2500, 5000, 500)] + [d for d in range(5000, 11000, 1000)]

In [ ]:
Index   = []
Storage = []

for Deposit in InitDepos:
    for Strk2Spt in Strik2Spot:
        for Cost in AvgCost:
            for Limit in CashLimit:
                for Withdraws in [True, False]:
                    
                    Index.append([Deposit, Strk2Spt, Cost, Limit, Withdraws])
                    Storage.append(MonteCarlo(mu, sd, Samples=Samples, Len=Len, Base=Base,
                                               Withdraw=Withdraws, CashLimit=Limit, CashReset=Limit,  AvgS2S=Strk2Spt, AvgCost=Cost, InitDep=Deposit).describe())
                    

In [ ]:
df = pd.concat(Storage, axis=0)
df.head(3).append(df.tail())

MonteCarlo(mu, sd, Samples=1000, Len=252, Base=3500, 
               Withdraw=False, CashLimit=75000, CashReset = 25000, AvgCost=50, AvgS2S=0, InitDep=250)